In [1]:
% matplotlib inline

import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
H1_data = np.genfromtxt('H1_times.csv')
L1_data = np.genfromtxt('L1_times.csv')

In [3]:
print(len(H1_data[np.where(H1_data[:,9] == 1)]))
print(len(H1_data[np.where(H1_data[:,10] == 1)]))
print(len(H1_data[np.where(H1_data[:,11] == 1)]))

print(len(L1_data[np.where(L1_data[:,9] == 1)]))
print(len(L1_data[np.where(L1_data[:,10] == 1)]))
print(len(L1_data[np.where(L1_data[:,11] == 1)]))

20
1
0
1
2
0


In [4]:
# using multilabels, with i = 0 = cbc,
#                           = 1 = burst,
#                           = 2 = stoch,
#                           = 3 = terrestrial
training = np.array([])
target = np.array([])
n_terr = 0
for event in H1_data:
    if (event[9] == 1 or event[10] == 1 or event[11] == 1):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            target = np.array([int(event[9]),int(event[10]),int(event[11]),0])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            target = np.vstack((target,np.array([int(event[9]),int(event[10]),int(event[11]),0])))
        
        continue # not counting hardware injections as terrestrial

    
    b = event[0] - 0.1
    e = event[1] + 0.1
    
    # find H1 events whose beginings and ends do not overlap with L1 events
    b_novr = len(np.where((b > L1_data[:,0]) & (b < L1_data[:,1]))[0]) == 0
    e_novr = len(np.where((e > L1_data[:,0]) & (e < L1_data[:,1]))[0]) == 0
    
    if (b_novr and e_novr and n_terr < 200):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            target = np.array([0,0,0,1])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            target = np.vstack((target, np.array([0,0,0,1])))
        n_terr += 1
        
n_terr = 0            
for event in L1_data:
    if (event[9] == 1 or event[10] == 1 or event[11] == 1):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            target = np.array([int(event[9]),int(event[10]),int(event[11]),0])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            target = np.vstack((target,np.array([int(event[9]),int(event[10]),int(event[11]),0])))
        
        continue # not counting hardware injections as terrestrial

    
    b = event[0]
    e = event[1]
    
    # find H1 events whose beginings and ends do not overlap with L1 events
    b_novr = len(np.where((b > H1_data[:,0]) & (b < H1_data[:,1]))[0]) == 0
    e_novr = len(np.where((e > H1_data[:,0]) & (e < H1_data[:,1]))[0]) == 0
    
    if (b_novr and e_novr and n_terr < 200):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            target = np.array([0,0,0,1])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            target = np.vstack((target, np.array([0,0,0,1])))
        n_terr += 1

In [5]:
# scale data for mlp
scaler = StandardScaler()  

scaler.fit(training)
train_scale = scaler.transform(training)

In [6]:
# split for training and testing
dat_train, dat_test, tar_train, tar_test = train_test_split(train_scale, target, test_size=0.4)

In [7]:
# MLP params
hidden = (3,)
activ = 'logistic'
solver = 'lbfgs'
lrn = 'adaptive'

In [8]:
mlp = MLPClassifier()
parameters = {'hidden_layer_sizes':[(3,)],'activation':['logistic'],'solver':['lbfgs'],'learning_rate':['adaptive'],'alpha':(10.0 ** -np.arange(1, 7))}
clf = GridSearchCV(mlp, parameters)
clf.fit(dat_train,tar_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-01,   1.00000e-02,   1.00000e-03,   1.00000e-04,
         1.00000e-05,   1.00000e-06]), 'activation': ['logistic'], 'solver': ['lbfgs'], 'learning_rate': ['adaptive'], 'hidden_layer_sizes': [(3,)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [9]:
alpha = clf.best_params_['alpha']

mlp = MLPClassifier(hidden_layer_sizes=hidden,activation=activ,solver=solver,learning_rate=lrn,alpha=alpha)
mlp.fit(dat_train,tar_train)

MLPClassifier(activation='logistic', alpha=9.9999999999999995e-07,
       batch_size='auto', beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(3,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [10]:
mlp.score(dat_test,tar_test)

0.90000000000000002

In [11]:
scores = cross_val_score(mlp, train_scale, target, cv=4)

In [12]:
print("Accuracy: %0.2f +/- %0.2f" % (scores.mean(), scores.std() * 2))
print(scores)

Accuracy: 0.92 +/- 0.06
[ 0.9245283   0.88679245  0.91509434  0.97169811]


In [13]:
print(np.sum(target, axis=0))

[ 21   3   0 400]


In [14]:
H1 = np.array([H1_data[:,2],H1_data[:,3],H1_data[:,4],H1_data[:,5]]).T
L1 = np.array([L1_data[:,2],L1_data[:,3],L1_data[:,4],L1_data[:,5]]).T
print(H1.shape)
print(L1.shape)
print(train_scale.shape)

(737, 4)
(5091, 4)
(424, 4)


In [15]:
H1_labels = mlp.predict(H1)
L1_labels = mlp.predict(L1)

In [16]:
print(np.sum(H1_labels,axis=0))
print(np.sum(L1_labels,axis=0))

[  0   0   0 737]
[   0    0    0 5091]
